<a href="https://colab.research.google.com/github/JakeOh/202505_BD50/blob/main/lab_da/da12_shape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrame 모양(shape) 변경

*   wide (columns) --> long (rows)
*   long (rows) --> wide (columns)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# stack vs unstack

In [3]:
df = pd.DataFrame(data=np.arange(1, 7).reshape((2, 3)),
                  columns=['a', 'b', 'c'],
                  index=['X', 'Y'])
df

,a,b,c
X,1,2,3
Y,4,5,6


In [4]:
df_stacked = df.stack()
df_stacked  #> 컬럼 이름들이 index(row label)로 바뀜. wide --> long

X  a    1
   b    2
   c    3
Y  a    4
   b    5
   c    6
dtype: int64

In [6]:
df_unstacked = df_stacked.unstack()
df_unstacked  #> 가장 마지막 레벨의 인덱스를 컬럼으로 변환. long --> wide

,a,b,c
X,1,2,3
Y,4,5,6


In [8]:
df_stacked.unstack(level=0)

,X,Y
a,1,4
b,2,5
c,3,6


컬럼 이름(인덱스)가 MultiIndex인 경우

In [11]:
df = pd.DataFrame(data=np.arange(1, 13).reshape((2, 6)),
                  columns=[['Lunch'] * 3 + ['Dinner'] * 3,
                           ['Fri', 'Sat', 'Sun'] * 2])
df

Lunch         Dinner        
    Fri Sat Sun    Fri Sat Sun
0     1   2   3      4   5   6
1     7   8   9     10  11  12

In [12]:
df.columns  #> MultiIndex

MultiIndex([( 'Lunch', 'Fri'),
            ( 'Lunch', 'Sat'),
            ( 'Lunch', 'Sun'),
            ('Dinner', 'Fri'),
            ('Dinner', 'Sat'),
            ('Dinner', 'Sun')],
           )

In [13]:
df.stack()  #> 가장 마지막 레벨의 컬럼 이름들을 index(row label)로 변환

Lunch  Dinner
0 Fri      1       4
  Sat      2       5
  Sun      3       6
1 Fri      7      10
  Sat      8      11
  Sun      9      12

In [15]:
df.stack(level=0)

Fri  Sat  Sun
0 Dinner    4    5    6
  Lunch     1    2    3
1 Dinner   10   11   12
  Lunch     7    8    9

In [17]:
df.stack(level=[0, 1])

0  Dinner  Fri     4
           Sat     5
           Sun     6
   Lunch   Fri     1
           Sat     2
           Sun     3
1  Dinner  Fri    10
           Sat    11
           Sun    12
   Lunch   Fri     7
           Sat     8
           Sun     9
dtype: int64

# pivot vs melt

In [18]:
df = pd.DataFrame(data={
    'time': ['Lunch'] * 3 + ['Dinner'] * 3,
    'day': ['Fri', 'Sat', 'Sun'] * 2,
    'tip': np.arange(1, 7),
    'total_bill': np.arange(10, 70, 10)
})
df

,time,day,tip,total_bill
0,Lunch,Fri,1,10
1,Lunch,Sat,2,20
2,Lunch,Sun,3,30
3,Dinner,Fri,4,40
4,Dinner,Sat,5,50
5,Dinner,Sun,6,60


## pivot

`pd.DataFrame.pivot()` 메서드 파라미터:
*   columns: pivoting 데이터프레임에서 컬럼 이름으로 사용하기 위한 컬럼(들)의 이름.
*   index: pivoting 데이터프레임에서 인덱스(row label)로 사용하기 위한 컬럼(들)의 이름.
*   values: pivoting 데이터프레임의 각 셀에 채울 값들을 가지고 있는 컬럼(들)의 이름.

In [19]:
df.pivot(columns='day', index='time', values='tip')

day,Fri,Sat,Sun
time,,,
Dinner,4,5,6
Lunch,1,2,3


In [24]:
df.pivot(columns='time', index='day', values='total_bill')

time,Dinner,Lunch
day,,
Fri,40,10
Sat,50,20
Sun,60,30
